# 11. 인터페이스 : 프로토콜에서 ABC까지

## Index
- 11.7 ABC의 정의와 사용
- 11.8 Tombola 서브클래스 테스트 방법
- 11.9 register()의 실제 용법
- 11.10 오리처럼 행동할 수 있는 거위
- 11.11 요약

## Remind
__ABC: Abstract Based Class__

## 11.7 ABC의 정의와 사용

상황가정: 광고가 전부 보여지기 전에는 중복 없이 랜덤으로 노출하는 광고 관리 framework ADAM을 만든다고 가정.

**추상 메서드**
- .load(...): 컨테이너에 아이템 넣기
- .pick(): 컨테이너에서 랜덤으로 한 개의 아이템을 제거하고 리턴

**구체 메서드**
- .loaded(): 컨테이너에 적어도 하나의 아이템으 있으면 True 리턴
- .inspect(): 내용을 변경하지 않고, 컨테이너에 있는 아이템들의 정렬된 tuple을 리턴

In [1]:
#Example 11-9
import abc

class Tombola(abc.ABC):
    
    @abc.abstractclassmethod #decorator for abstract method
    def load(self, iterable):
        """Add item from an iterable."""
        
    @abc.abstractclassmethod
    def pick(self):
        """Remove item at random, returning it.
        
        This method should raise 'LookupError' when the instance is empty.
        """
        
    def loaded(self):
        """Return 'True' if there's at least 1 item, 'False' otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

ABC 안에서 구체 메서드를 제공하는 것이 가능하다.<br>`.inspect()`는 서브 클래스에서 더 좋은 구현을 하겠지만 그럴 필요가 없다.(이미 구현되어 있기 때문)<br>`.loaded()`는 `bool()`에 적용하려고 정렬된 tuple을 만들기 때문에 비싼 연산이다. 따라서 서브 클래스에서 더 잘 구현해야 할 것이다.<br><br>
`LookupError`는 `IndexError`와 `KeyError`보다 상위 계층의 Error이기 때문에 둘 다 잡을 수 있다!

In [2]:
#Example 11-11
#from tombola import Tombola
class Fake(Tombola):
    def pick(self):
        return 13

In [3]:
Fake
#나는 왜 더 안뜨지...

__main__.Fake

In [4]:
f = Fake()
#대충 구체화 다 안하면 객체화 할 수 없다는 말
#load()는...? 구체화 안할거야....??????

TypeError: Can't instantiate abstract class Fake with abstract methods load

### ABC 문법 디테일(직독직해)
`abc.ABC` 같이 편한 건 Python 3.4나 되서야 나왔다. 그 전의 버전에서는 어떻게 ABC를 선언할까?

일단,<br>
1. Python 3.4 이상: 그냥 `abc.ABC` 쓰면 됨
2. Python 3.4 미만의 Python 3: 
```Python
class Tombola(metaclass=abc.ABCMeta):
```
이렇게 `metaclass` 키워드 인자를 사용하면 된다.
3. Python 2
```Python
class Tomola(object):
    __metaclass__ = abc.ABCMeta
```
이렇게 `__metaclass__` 를 사용하면 된다. (`__metaclass__`는 Chapter 21에서 다시 나온다)

<br>
원래 `@abstractclassmethod`, `@abstractstaticmethod`, `@abstractproperty` 이렇게 3개가 더 있는데 Python 3.3을 기준으로 사용하지 않는다. stack decorator를 지원하기 때문!

```Python
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass
```
이런 식으로..

### Subclassing Tombola ABC
주어진 Tombola ABC, 우리는 지금 발전한다 두개의 구체적인 서브클래스들 그것은 만족한다 그것의 인터페이스를.

In [5]:
#Example 11-12
import random

#from tombola import Tombola


class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

`BingoCage`는 비싼 `loaded`와 멍청한 `inspect`를 상속한다. 여튼 구현을 다 하지 않아도 ABC에서 구현된 메소드가 있으면 사용해도 된다. 느리고 비효율적일뿐..! 정확한 결과는 제공한다.

In [6]:
#Example 11-13
import random

#from tombola import Tombola


class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

### A Virtual Subclass of Tombola
Goose Typing의 필수적인 특성은 ABC로부터 아무것도 상속받지 않더라도 ABC의 가상 서브 클래스로 등록할 수 있다는 것이다. 가상 서브클래스로 등록하면 Python은 묻지도 따지지도 않고 우리를 믿어준다.<br>
Goose Typing?

In [1]:
#Example 11-14
from random import randrange

#from tombola import Tombola

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    #Tombolist.load is the same as list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
#Tombola.register(TomboList) #Legacy

NameError: name 'Tombola' is not defined

In [8]:
issubclass(TomboList, Tombola)

True

In [9]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [10]:
TomboList.__mro__ #Method Resolution Order
#Tombola가 없다!

(__main__.TomboList, list, object)

## 11.8 Tombola 서브클래스 테스트 방법
- `__subclasses__()`: 클래스의 서브 클래스의 리스트를 리턴. 가상 서브클래스는 포함되지 않음.
- `_abc_registry`: 추상 클래스의 가상 서브 클래스를 등록하기 위한 약한 참조 데이터 속성

In [ ]:
#Example 11-15
import doctest

#from tombola import Tombola

#module to test
#import bingo, lotto, tombolist, drum

TEST_FILE = 'tombola_test.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

def main(argv):
    verbose = '-v' in argv
    real_subclasses = Tombola.__subclasses__()
    virtual_subclasses = list(Tombola._abc_registry)
    
    for cls in real_subclasses + virtual_subclasses:
        test(cls, verbose)
        
def test(cls, verbose=False):
    
    res = doctest.testfile(
            TEST_FILE,
            verbose=verbose,
            optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))
    

if __name__ == '__main__':
    import sys
    main(sys.argv)

## 11.9 register()의 실제 용법
Example 11-14 마지막 줄의 comment.<br>
```Python
Tombola.register(TomboList)
```
Decorator가 잘 작동하지만, 위의 코드와 같이 사용하는 용례가 많다. Built-in 타입인 `tuple`,`str`,`range`,`memoryview`가 `Sequence`의 가상 서브클래스로 등록되어 있다.
```Python
Sequence.register(tuple)
Sequence.register(str)
Sequence.register(range)
Sequence.register(memoryview)
```

## 11.10 오리처럼 행동할 수 있는 거위

In [11]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [12]:
issubclass(Struggle, abc.Sized)

True

음..? 가상 서브클래스로 등록하지 않았는데도 서브클래스로 인식한다. 이것은 `abc.Sized`가 `__subclasshook__`이라는 특수 클래스 메소드를 구현했기 때문이다.

In [13]:
#Example 11-17
from abc import *

class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

관심이 있다면 `ABCMeta.__subclasscheck__` 메소드를 보라는데 많은 if와 두번의 재귀가 있다네요. 전 pass!<br>
여튼, `__subclasshook__`은 goose typing에 duck typing의 DNA를 추가합니다. 그래서 우리의 ABC에서 쓰는 게 좋을까? 필자가 본 `__subclasshook__`은 특별한 상태에서만 구현되어 있고, 이 특수한 상황조차도 위험한 가정을 동반하며, 부정적인 느낌..?

## 11.11 요약
### Keywords
- interface(like protocol)
    - Nothing to do with inheritance.
    - Duck Typing
- ABC(Abstract Based Class)
    - Goose Typing
    - Make interfaces explicit and classes may claim to implement and interface by subclassing an ABC or by registering with it.